<a href="https://colab.research.google.com/github/olorunfemibabalola/Bias-Detection-NLP/blob/main/Policy_Bias_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
'''
Import modules
Import the dataset
Perform exploratory data analysis
Clean the data
Split into training and testing sets
Create a model
Train the model
Make predictions
Test the model
Evaluate the model
Make predictions on new data
Persist the model for future use
Load a persisted model
Make predictions on new data
'''

'\nImport modules\nImport the dataset\nPerform exploratory data analysis\nClean the data\nSplit into training and testing sets\nCreate a model\nTrain the model\nMake predictions\nTest the model\nEvaluate the model\nMake predictions on new data\nPersist the model for future use\nLoad a persisted model\nMake predictions on new data\n'

--------------------------------------------------------------------------------



In [7]:
!pip install -q transformers torch PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fileinput
from transformers import pipeline

In [10]:
from google.colab import files

userChoice = input("Woud you like to input a text or a file? (text/file)").lower().strip()
while (userChoice != "text") and (userChoice != "file"):
    print("Invalid input. Please enter 'text' or 'file'")
    userChoice = input("Woud you like to input a text or a file? (text/file)").lower().strip()
else:
    print(f"You have selected {userChoice}")

if userChoice == "text":
  userText = input("Enter your text:")
else:
  print("Upload your file below:")
  #This creates the "Upload" button
  uploadedFile = files.upload()

  # The 'uploaded' variable is a dictionary:
  # Key = filename, Value = file data (bytes)
  for filename in uploadedFile.keys():
    print(f'User uploaded file "{filename}" with a length of {len(uploadedFile[filename])} bytes')




Woud you like to input a text or a file? (text/file)text
You have selected text
Enter your text:hvv


In [12]:
from PyPDF2 import PdfReader

# Define the PDF file name
# The filename variable already holds the correct name from previous steps

# Open the PDF file in binary read mode
reader = PdfReader(filename)

# Initialize an empty string to store the extracted text
user_file= ""

# Iterate through each page and extract text
for page in reader.pages:
    user_file += page.extract_text()

# Display a portion of the extracted text to verify
print(user_file[:1000])
print(f"Total characters extracted: {len(user_file)}")

NameError: name 'filename' is not defined

In [13]:
print(user_file)

NameError: name 'user_file' is not defined

In [14]:
user_file.lower()
new = user_file.split(".")

NameError: name 'user_file' is not defined

In [ ]:
# !pip install -q transformers torch

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Step 1: Initialize the Auditor (RoBERTa Encoder)
# Specialized for detecting social and cognitive biases
auditor = pipeline("text-classification",
                   model="valurank/distilroberta-bias",
                   device=0 if torch.cuda.is_available() else -1)

# Step 2: Initialize the Conversationalist (GPT-2 Decoder)
chat_model_name = "openai-community/gpt2"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_model_name)
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name)

def get_chatbot_response(prompt):
    # Perform Bias Audit FIRST
    audit_result = auditor(prompt)[0]

    # If Bias is detected above a 70% confidence threshold
    if audit_result['label'] == 'Biased' and audit_result['score'] > 0.7:
        return f"⚠️ [BIAS ALERT]: I detected potential {audit_result['label']} in your prompt. " \
               f"Please rephrase to be more inclusive."

    # Otherwise, generate a standard response
    inputs = chat_tokenizer.encode(prompt + chat_tokenizer.eos_token, return_tensors='pt')
    outputs = chat_model.generate(inputs, max_length=100, do_sample=True, top_k=50, top_p=0.95)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 3: Interactive Chat Loop
print("🛡️ Inclusive Assistant Active. Type 'quit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']: break

    response = get_chatbot_response(user_input)
    print(f"Bot: {response}\n")


Device set to use cpu


🛡️ Inclusive Assistant Active. Type 'quit' to stop.
You: hey


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: heyYou may have seen a great movie or heard from a great actor and a great story you can enjoy at any time. But how do you get to it? You just need to know how.

A couple of things you need to know about AIM Aims:

AIM Aims is a toolkit on their site to help you take a hard look at what to watch and don't watch at certain times of year. I've used it for my work to see

You: i hate women


In [16]:
# Ensure you have the necessary libraries installed
# !pip install -q PyPDF2 gradio transformers torch

import PyPDF2
import gradio as gr

def extract_text_from_pdf(pdf_file):
    """Extracts text from an uploaded PDF file object."""
    reader = PyPDF2.PdfReader(pdf_file.name)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + " "
    return text

def audit_document(file):
    """Scans the entire document and returns biased sentences."""
    if file is None:
        return "No file uploaded."

    text = extract_text_from_pdf(file)
    sentences = text.split('.')
    biased_findings = []

    for sentence in sentences:
        clean_sentence = sentence.strip()
        if len(clean_sentence) > 10: # Filter out noise
            result = auditor(clean_sentence)[0]
            if result['label'] == 'Biased' and result['score'] > 0.75:
                biased_findings.append(f"• \"{clean_sentence}\" (Confidence: {result['score']:.2f})")

    if not biased_findings:
        return "✅ No biased statements found. The document is safe and inclusive."
    else:
        report = "⚠️ Potential bias detected in the following statements:\n\n" + "\n".join(biased_findings)
        return report

# Updated Chatbot Logic with "Document Upload" Intent
def chatbot_response(message, history):
    message_lower = message.lower()

    # Feature 1: Check for Document Intent
    if any(word in message_lower for word in ["upload", "document", "file", "scan"]):
        return "Sure! Please use the **Upload File** button below to provide your document for a bias audit."

    # Feature 2: Standard Chat Audit
    res = auditor(message)[0]
    if res['label'] == 'Biased':
        return f"⚠️ BIAS ALERT: I detected potential {res['label']} in your input. Try using neutral phrasing."

    return "✅ Inclusive input. I am ready to help with your policy questions."

# UI Layout (Using Gradio Blocks for better control)
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🛡️ Inclusive Policy Assistant & Auditor")

    with gr.Tab("Chat"):
        gr.ChatInterface(chatbot_response)

    with gr.Tab("Document Audit"):
        file_input = gr.File(label="Upload Corporate Policy (PDF)")
        audit_output = gr.Textbox(label="Audit Report", lines=10)
        audit_button = gr.Button("Run Audit")
        audit_button.click(audit_document, inputs=file_input, outputs=audit_output)

demo.launch(debug=True)


/tmp/ipython-input-2350190090.py:55: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft") as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6186570618f502c488.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6186570618f502c488.gradio.live
